# Iris Case Study: Encoding string target variables

## Load Libraries

In [2]:
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Load Data

In [4]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
data = read_csv(url, header = None)

## Split data into X/Y

In [5]:
X = data.values[:,0:4]
Y = data.values[:,4]

## Encode String Class (target variable) as integers

In [6]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_y = label_encoder.transform(Y)

## Split into train/test sets

In [7]:
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(X, label_encoded_y,
                                                    test_size = 0.33, 
                                                    random_state=seed)

## Fit Model on Training Set

In [8]:
model = XGBClassifier()
model.fit(X_train, Y_train)
print(model)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


## Make predictions and evaluate

In [12]:
#Predict
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#Evaluate
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.00%


# Breast Cancer Case Study: categorical datasets
Some datasets only contain categorical data, for example the breast cancer dataset. This dataset describes the technical details of breast cancer biopsies and the prediction task is to predict whether or not the patient has a recurrence of cancer, or not.

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer-data'
data = 